#### import necessary libraries

In [1]:
import numpy as np
from scipy import linalg
import random
import pandas as pd
from numpy import linalg as LA
from scipy.linalg import cho_factor, cho_solve

#### Import helper_function

In [2]:
import helper_function as hf

#### Driver Code 

In [3]:
def main_func(arr,eps):
    order = []
    (GEPP1,GERP1,GECP1,CHOP1,LU1,CHOLESKY1) = ([],[],[],[],[],[])
    (GEPP2,GERP2,GECP2,CHOP2,LU2,CHOLESKY2) = ([],[],[],[],[],[])
    
    for n in arr:
        A,b = hf.construct(n,eps)
        order.append(n)

        print(f"                            Order of A = {n}\n")
        if hf.GEPP(A,eps) == None : 
            (diff1,diff2) = (None,None)
            print("1 - GEPP , Terminated @ GEPP ")
        else:
            P,L,U = hf.GEPP(A,eps)
            X = hf.subs_partial(P,L,U,b,n)
            diff1,diff2 = hf.norm_calc(P,A,L,U,X,b)
            print(f"1 - GEPP , norm (PA-LU) = {diff1}, norm (AX - b) = {diff2} ")
        GEPP1.append(diff1)
        GEPP2.append(diff2)
        
        if hf.GERP(A,eps) == None : 
            (diff1,diff2) = (None,None)
            print("2 - GERP , Terminated @ GERP ")
        else:
            P,L,U,Q = hf.GERP(A,eps)
            X = hf.subs_rest_all(P,L,U,b,Q,n)
            diff1,diff2 = hf.norm_calc_rest(X,P,A,Q,L,U,b)
            print(f"2 - GERP ,norm (PAQ-LU) = {diff1}, norm (AX - b) = {diff2} ")
        GERP1.append(diff1)
        GERP2.append(diff2)

        if hf.GECP(A,eps) == None :
            (diff1,diff2) = (None,None) 
            print("3 - GECP , Terminated @ GECP ")
        else:
            P,L,U,Q = hf.GECP(A,eps)
            X = hf.subs_rest_all(P,L,U,b,Q,n)
            diff1,diff2 = hf.norm_calc_rest(X,P,A,Q,L,U,b)
            print(f"3 - GECP , norm (PAQ-LU) = {diff1}, norm (AX - b) = {diff2} ")
        GECP1.append(diff1)
        GECP2.append(diff2)

        if hf.CHOP(A,eps) == None :
            (diff1,diff2) = (None,None) 
            print("4 - CHOP , Terminated @ CHOP ")
        else:
            P,L,Q = hf.CHOP(A,eps)
            X = hf.subs_rest_all(P,L,L.T,b,Q,n)
            diff1,diff2 = hf.norm_calc_rest(X,P,A,Q,L,L.T,b)
            print(f"4 - CHOP , norm (PAQ-LU) = {diff1}, norm (AX - b) = {diff2} ")
        CHOP1.append(diff1)
        CHOP2.append(diff2)  
    
        P,L,U = linalg.lu(A)
        lu,piv = linalg.lu_factor(A)
        X = linalg.lu_solve((lu, piv), b)
        (diff1,diff2) = (linalg.norm(P.T@A - L@U), linalg.norm(A@X-b,ord = 2))
        print(f"5 - Scipy LU , norm (PA-LU) = {diff1}, norm (AX - b) = {diff2} ")  
        LU1.append(diff1)
        LU2.append(diff2)

        L_ = linalg.cholesky(A,lower=True)
        C,L = linalg.cho_factor(A)
        X = linalg.cho_solve((C,L), b)
        (diff1,diff2) = (linalg.norm(A - L_@L_.T), linalg.norm(A@X-b,ord=2))
        print(f"6 - Scipy Cholesky, norm (A-LL(trans)) = {diff1}, norm (AX - b) = {diff2} ")
        CHOLESKY1.append(diff1)
        CHOLESKY2.append(diff2)

        print('_'*100)


    norm_PA = {
    'Order' :  order,
    'GEPP' : GEPP1, 
    'GERP': GERP1,
    'GECP' : GECP1,
    'CHOP' : CHOP1,
    'LU' :LU1, 
    'CHOLESKY' : CHOLESKY1
    }

    norm_AX = {
        'Order' :  order,
        'GEPP' : GEPP2, 
        'GERP': GERP2,
        'GECP' : GECP2,
        'CHOP' : CHOP2,
        'LU' :LU2, 
        'CHOLESKY' : CHOLESKY2
    }

    df_norm_AX = pd.DataFrame(norm_AX)
    df_norm_PA = pd.DataFrame(norm_PA)
    return df_norm_PA,df_norm_AX
    

In [4]:
arr = input().rstrip()
arr = list(map(int,arr.split(" ")))

In [5]:
eps = 1e-10
df_norm_PA,df_norm_AX = main_func(arr,eps)

                            Order of A = 7

1 - GEPP , norm (PA-LU) = 6.049187461416408e-17, norm (AX - b) = 1.737759081973225e-07 
2 - GERP ,norm (PAQ-LU) = 5.1925927263190304e-17, norm (AX - b) = 2.8115454709698567e-07 
3 - GECP , norm (PAQ-LU) = 6.206335383118183e-17, norm (AX - b) = 1.1151007970493857e-07 
4 - CHOP , norm (PAQ-LU) = 3.671717528720129e-17, norm (AX - b) = 7.300048299977713e-08 
5 - Scipy LU , norm (PA-LU) = 5.1925927263190304e-17, norm (AX - b) = 1.7631291696726132e-07 
6 - Scipy Cholesky, norm (A-LL(trans)) = 3.3993498887762956e-17, norm (AX - b) = 1.7881393432617188e-07 
____________________________________________________________________________________________________
                            Order of A = 8

1 - GEPP , norm (PA-LU) = 6.655553224256149e-17, norm (AX - b) = 2.132480599880018e-06 
2 - GERP ,norm (PAQ-LU) = 4.8074067159589095e-17, norm (AX - b) = 3.4385216478958026e-06 
3 - GECP , norm (PAQ-LU) = 3.925231146709438e-17, norm (AX - b) = 1.651812369

##### Norm Table of (PA - LU)

In [6]:
df_norm_PA

,Order,GEPP,GERP,GECP,CHOP,LU,CHOLESKY
0,7,6.049187e-17,5.192593e-17,6.206335e-17,3.671718e-17,5.192593e-17,3.399350e-17
1,8,6.655553e-17,4.807407e-17,3.925231e-17,4.602742e-17,5.374844e-17,4.807407e-17
2,9,6.973502e-17,9.714451e-17,NaN,5.419449e-17,6.546139e-17,5.969062e-17
3,10,7.505561e-17,1.129570e-16,NaN,5.419449e-17,7.211110e-17,6.359601e-17
4,11,NaN,1.273812e-16,NaN,6.321633e-17,7.505561e-17,7.376145e-17
5,12,NaN,1.447221e-16,NaN,6.727507e-17,7.343435e-17,6.691627e-17
6,13,NaN,1.712375e-16,NaN,7.076311e-17,8.092071e-17,6.938894e-17


##### Norm Table of (AX - b)

In [7]:
df_norm_AX

,Order,GEPP,GERP,GECP,CHOP,LU,CHOLESKY
0,7,1.737759e-07,2.811545e-07,1.115101e-07,7.300048e-08,1.763129e-07,1.788139e-07
1,8,2.132481e-06,3.438522e-06,1.651812e-06,2.336015e-06,1.651812e-06,3.015783e-06
2,9,8.113615e-05,2.488624e-04,NaN,7.129369e-05,1.145601e-04,9.505247e-05
3,10,3.548028e-02,3.653962e-02,NaN,2.415882e-02,3.232997e-02,2.870496e-02
4,11,NaN,4.871393e-01,NaN,2.724312e-01,2.633172e-01,2.670001e-01
5,12,NaN,1.328533e+02,NaN,4.884670e+01,6.081118e+01,4.718050e+01
6,13,NaN,6.880320e+02,NaN,6.331206e+02,4.119999e+02,2.570440e+03


##### Showing results when no epsilon condition is imposed. It will terminate only when pivot is 0 .

In [8]:
eps = 0
df_norm_PA_zero,df_norm_AX_zero = main_func(arr,eps)

                            Order of A = 7

1 - GEPP , norm (PA-LU) = 6.049187461416408e-17, norm (AX - b) = 3.95372484964776e-07 
2 - GERP ,norm (PAQ-LU) = 5.1925927263190304e-17, norm (AX - b) = 2.08531089836787e-06 
3 - GECP , norm (PAQ-LU) = 6.206335383118183e-17, norm (AX - b) = 7.251217997429632e-07 
4 - CHOP , norm (PAQ-LU) = 3.671717528720129e-17, norm (AX - b) = 4.76837158203125e-07 
5 - Scipy LU , norm (PA-LU) = 5.1925927263190304e-17, norm (AX - b) = 7.44461774635124e-07 
6 - Scipy Cholesky, norm (A-LL(trans)) = 3.3993498887762956e-17, norm (AX - b) = 4.6169559314339364e-07 
____________________________________________________________________________________________________
                            Order of A = 8

1 - GEPP , norm (PA-LU) = 6.655553224256149e-17, norm (AX - b) = 2.412626388678268e-05 
2 - GERP ,norm (PAQ-LU) = 4.8074067159589095e-17, norm (AX - b) = 0.0001113473474069704 
3 - GECP , norm (PAQ-LU) = 3.925231146709438e-17, norm (AX - b) = 2.1579186437577746e

##### Norm Table of (PA - LU)

In [9]:
df_norm_PA_zero

,Order,GEPP,GERP,GECP,CHOP,LU,CHOLESKY
0,7,6.049187e-17,5.192593e-17,6.206335e-17,3.671718e-17,5.192593e-17,3.399350e-17
1,8,6.655553e-17,4.807407e-17,3.925231e-17,4.602742e-17,5.374844e-17,4.807407e-17
2,9,6.973502e-17,9.714451e-17,7.726825e-17,5.419449e-17,6.546139e-17,5.969062e-17
3,10,7.505561e-17,1.129570e-16,8.722054e-17,5.419449e-17,7.211110e-17,6.359601e-17
4,11,7.881069e-17,1.273812e-16,9.886402e-17,6.321633e-17,7.505561e-17,7.376145e-17
5,12,8.993834e-17,1.447221e-16,1.026863e-16,6.727507e-17,7.343435e-17,6.691627e-17
6,13,8.993834e-17,1.712375e-16,7.972182e-17,7.076311e-17,8.092071e-17,6.938894e-17


##### Norm Table of (AX - b)

In [10]:
df_norm_AX_zero

,Order,GEPP,GERP,GECP,CHOP,LU,CHOLESKY
0,7,3.953725e-07,0.000002,7.251218e-07,4.768372e-07,7.444618e-07,4.616956e-07
1,8,2.412626e-05,0.000111,2.157919e-05,2.412626e-05,2.288818e-05,2.642900e-05
2,9,4.111967e-04,0.000847,3.222898e-04,3.062947e-04,2.903981e-04,2.788106e-04
3,10,5.281569e-03,0.009200,4.055969e-03,5.167483e-03,7.453290e-03,6.605346e-03
4,11,2.628532e-01,0.311718,2.025231e-01,1.848775e-01,1.815461e-01,1.808725e-01
5,12,5.830952e+00,10.862780,7.348469e+00,5.477226e+00,6.782330e+00,3.741657e+00
6,13,6.171854e+00,10.087194,5.854177e+00,1.374063e+01,9.584277e+00,2.655604e+01


#### Condition Number bound :-

We know that, given a problem Ax = b (we need to find x), with small perturbations in 'b' resulting in 'delta_b', the following inequality holds <br>
 <center> norm(delta_x) / norm(x)  <= K(A)* (norm(delta_b) / norm(b))  </center>  where delta_x is the solution for the problem Ax = delta_b and K(A) is the condition number of A. <br>
Therefore,<br>
    <center> (norm(delta_x)*norm(b))/(norm(x)*norm(delta_b)) <= K(A) </center> 

*Conclusion* : As we increase the size of matrix i.e n, K(A) increases. 

In [11]:
print("Lower bounds for Condtion no. for diff sizes i.e. n")
for n in range(5,1000,50):
    A,c =hf.construct(n,0)
 
    b=np.random.rand(n,1)*1000
    b_delta=np.random.rand(n,1)*(1e-6)
    
    P,L,U = hf.GEPP(A,0)
    x = hf.subs_partial(P,L,U,b,n)   

    b_dash = b + b_delta
    x_delta = hf.subs_partial(P,L,U,b_dash, n)    
    
    print( f"order n = {n} .... {(linalg.norm(x_delta-x,ord=2)*linalg.norm(b,ord=2))/(linalg.norm(b_delta,ord=2)*(linalg.norm(x,ord=2)))}") 

Lower bounds for Condtion no. for diff sizes i.e. n
order n = 5 .... 2.056943876336424
order n = 55 .... 13.079717951534477
order n = 105 .... 563.9595598644866
order n = 155 .... 7201.75215473937
order n = 205 .... 16282.996657641239
order n = 255 .... 28000.4136244328
order n = 305 .... 44369.45154012346
order n = 355 .... 187784.32328661022
order n = 405 .... 80151.93750744889
order n = 455 .... 251861.4487392586
order n = 505 .... 239855.1402097235
order n = 555 .... 854778.4931364973
order n = 605 .... 575689.3530071353
order n = 655 .... 924385.8902992817
order n = 705 .... 533222.7700139055
order n = 755 .... 878396.7026624632
order n = 805 .... 850217.9977331477
order n = 855 .... 437503.04267593485
order n = 905 .... 679871.9837319724
order n = 955 .... 1629483.8021845869


#### Helper function 

In [12]:
import numpy as np
from scipy import linalg
import random
from numpy import linalg as LA

# done +1 +1 in i and j in order to make them or our requirements o/w it will have divide by 0 error
def construct(n,eps):
    b = [random.randint(-100,100) for i in range(n*1)]
    A = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            value = 1/(i+1+j+1-1)
            A[i,j] = value
    return A,b

# 1 - Partial Pivoting

def GEPP(A,eps): 
    
    A = A.astype(float)
    n = A.shape[0]
    U = A.copy()
    L = np.identity(n)
    P = np.identity(n)
    pivot = 0
    
    for k in range(n-1):
        # Below snippet does partial pivoting 
        column_k = np.abs(U[k:, k])
        pivot = max(column_k)
        if pivot <= eps  :
            return None
 
        index_col = np.argmax(column_k)
        
        if k != (index_col + k) :
            U[[k, index_col+k],:] = U[[index_col+k, k],:]
            P[[k, index_col+k],:] = P[[index_col+k, k],:]
            L[[k, index_col+k],:k] = L[[index_col+k, k],:k]
        
        # Now comes the computation part of LU
        # k denotes the no. of row and j denotes the column and i again columns
            
        for j in range(k+1,n):
            L[j,k] = U[j,k]/U[k,k]  
            U[j,k:] -= L[j,k]*U[k,k:]

    return P,L,U

# 2 - Rook Pivoting

def GERP(A,eps): 

    A = A.astype(float)
    n = A.shape[0]
    U = A.copy()
    L = np.identity(n)
    P = np.identity(n)
    Q = np.identity(n)

    for k in range(n-1):
        
        column_vals = np.abs(U[k:,k])   # k th column
        row_vals = np.abs(U[k,k:])      # k th row

        (row, col) = (-1,-1)

        (max_col,max_row) = (max(column_vals),max(row_vals))        # Finds postion of max element for pivoting 
        if max_col >= max_row and all([max_col,max_row]) >= eps :
            col = np.argmax(column_vals) + k
        elif all([max_col,max_row]) >= eps  :
            row = np.argmax(row_vals) + k
        else:
            return None
       
        P[[k,row],:] = P[[row,k],:]     # Logic : PAQ should be equal to LU where
        U[[k,row],:] = U[[row,k],:]     # P and Q are permutation matrices 
        
        Q[:,[k,col]] = Q[:,[col,k]]  
        U[:,[k,col]] = U[:,[col,k]]
        
        L[[k,row],:k] = L[[row,k],:k]
        
        for j in range(k + 1, n):
            L[j, k] = U[j, k] / U[k, k]
            U[j, k:] -= L[j, k] * U[k, k:]

    return P, L, U, Q

# 3 - Complete pivoting

def GECP(A,eps):
    
    A = A.astype(float)
    n = A.shape[0]
    U = A.copy()
    L = np.identity(n)
    P = np.identity(n)
    Q = np.identity(n)

    for k in range(n):
        max_index = np.unravel_index(np.argmax(np.abs(U[k:, k:])), (n-k, n-k))  # find max element/postion in n-k * n-k matrix
        
        row = max_index[0] + k
        col = max_index[1] + k

        if U[row,col] <= eps:
            return None
        
        P[[k,row],:] = P[[row,k],:]
        U[[k,row],:] = U[[row,k],:]
        
        Q[:,[k,col]] = Q[:,[col,k]]   
        U[:,[k,col]] = U[:,[col,k]]
        
        L[[k,row],:k] = L[[row,k],:k]
        
        for j in range(k + 1, n):
            L[j, k] = U[j, k] / U[k, k]
            U[j, k:] -= L[j, k] * U[k, k:]

    return P, L, U, Q

# 4 - Cholesky Decomposition 

def CHOP(A,eps):

    A = A.astype(float)
    n = A.shape[0]
    L = np.identity(n)
    P = np.identity(n)
    Q = np.identity(n)

    # Check for symmetry and positive definiteness (basic check)
    if not np.allclose(A, A.T):
        raise ValueError("Input matrix is not symmetric.")
    
    for k in range(n):

        if not np.all(np.linalg.eigvals(A[k:,k:]) > 0):    # checks for +ve definiteness for every submatrix  
            return P,L,Q

        pos = np.argmax([A[j,j] for j in range(k,n)]) + k
        if A[pos,pos] <= eps:
            print("Terminated @ CHOP")
            return None

        A[[k,pos],:] = A[[pos,k],:]
        P[[k,pos],:] = P[[pos,k],:]
        L[[k,pos],:] = L[[pos,k],:]

        A[:,[k,pos]] = A[:,[pos,k]]
        Q[:,[k,pos]] = Q[:,[pos,k]]  
        L[:,[k,pos]] = L[:,[pos,k]]
        
        L[k,k] = np.sqrt(A[k,k] - np.sum(L[k, :k] ** 2))
        for p in range(k+1,n):
            L[p,k] = (A[p,k] - np.sum(L[p,:k]*L[k,:k])) / L[k,k]
        
    return P,L,Q

# Calculates norms of (PA - LU) and (AX - b) respectively 

def norm_calc(P,A,L,U,X,b): # returns norms for Mat and vector
    X = X.astype(float)
    diff_mat = P @ A - L @ U
    diff_vector = A @ X - b
    diff_vector = diff_vector.astype(float)
    return np.linalg.norm(diff_mat), np.linalg.norm(diff_vector)

# forward substitution 

def forward_subs(P,L,b,n):
    Pb = P @ b
    Y = np.zeros(n)
    for i in range(n):
        temp = 0
        for j in range(i):
            temp += Y[j]*L[i,j]
        Y[i] = (Pb[i] - temp)/L[i,i]
    return Y

# backward substitution

def backward_subs(U,Y,n):
    X = np.ones(n)
    for i in range(n-1,-1,-1):
        temp = 0
        for j in range(n-1,i,-1):
            temp += X[j]*U[i,j]
        X[i] = (Y[i] - temp)/U[i,i]
    return X

# used for GEPP 

def subs_partial(P,L,U,b,n):
    Y = forward_subs(P,L,b,n)
    X = backward_subs(U,Y,n)
    return X

# used for GERP, GECP, CHOP 

def subs_rest_all(P,L,U,b,Q,n):
    Y = forward_subs(P,L,b,n)
    x_prime = backward_subs(U,Y,n)
    return Q @ x_prime                       # if needed can explain in person   

# used for calculating norm of partial pivoting method 

def norm_calc(P,A,L,U,X,b): 
    X = X.astype(float)
    diff_mat = P @ A - L @ U
    diff_vector = A @ X - b
    diff_vector = diff_vector.astype(float)
    return np.linalg.norm(diff_mat), np.linalg.norm(diff_vector)

# used for calculating norm of GERP, GECP, CHOP

def norm_calc_rest(X,P,A,Q,L,U,b):
    X = X.astype(float)
    diff_mat = P @ A @ Q - L @ U
    diff_vector = A @ X - b
    diff_vector = diff_vector.astype(float)
    return np.linalg.norm(diff_mat), np.linalg.norm(diff_vector)
